# ImageProcessing

This code is executed in Google Colab


In [ ]:
import os
import shutil
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# Copy the Original Image
file_path = '/content/drive/MyDrive/졸논이미지_추가2'
dst_path = '/content/drive/MyDrive/scaffold_dlc2'

shutil.copytree(file_path, dst_path)

## Renaming

Renaming the file name(2021XXXXXXX.jpg -> 001.jpg)

In [ ]:
file_name = os.listdir(dst_path)
file_name.sort()

# Numbering: 001, 002, 003 ...
idx = 1
for li in file_name:
    src = os.path.join(dst_path,li)
    dst = str(idx).zfill(3) + '.jpg'
    dst = os.path.join(dst_path, dst)
    os.rename(src, dst)
    idx += 1

## Crop Image

Crop Image to calculate micro-pit's hole using boundary which is made by higher threshold

In [ ]:
thresh = 20
img_path = '/content/drive/MyDrive/scaffold_dlc2/'
file_name = os.listdir(img_path)
file_name.sort()

file_idx = 1
for i in file_name:
    path = img_path + i
    img = cv2.imread(path)

    h, w = img.shape[:2] 
    h1, h2 = int(h * 0.05), int(h * 0.95) 
    w1, w2 = int(w * 0.05), int(w * 0.95) 
    newImg = img[h1: h2, w1: w2]

    # Remove noise using high threshold
    gray = cv2.cvtColor(newImg, cv2.COLOR_BGR2GRAY)
    img2 = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)[1]
    contours = cv2.findContours(img2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate The Image's Boundary
    x1 = [] #x_min
    y1 = [] #y_min
    x2 = [] #x_max
    y2 = [] #y_max
    for idx in  range(1, len(contours[0])): 
        ret = cv2.boundingRect(contours[0][idx]) 
        x1.append(ret[0]) 
        y1.append(ret[1]) 
        x2.append(ret[0] + ret[2]) 
        y2.append(ret[1] + ret[3])
    for idx in  range(1, len(contours[1])): 
        ret = cv2.boundingRect(contours[1][idx]) 
        x1.append(ret[0]) 
        y1.append(ret[1]) 
        x2.append(ret[0] + ret[2]) 
        y2.append(ret[1] + ret[3])

    for contour in contours:
        for idx in range(1, len(contour)):
            ret = cv2.boundingRect(contour[idx]) 
            x1.append(ret[0]) 
            y1.append(ret[1]) 
            x2.append(ret[0] + ret[2]) 
            y2.append(ret[1] + ret[3])

    # Padding
    delta = 210    
    x1_min = min(x1)+delta
    y1_min = min(y1)+delta + 50
    x2_max = max(x2)-delta
    y2_max = max(y2)-delta + 50

    file_idx += 1

    crop_img = newImg[y1_min:y2_max, x1_min:x2_max]

    # extracted image -> Binary Image
    binary_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(binary_img, thresh, 255, cv2.THRESH_BINARY)
    cv2.imwrite("/content/drive/MyDrive/scaffold_preprocess_dlc2/"+i,thresh1)



## Calculate the area of hole

Count the number of black pixel for each hole

In [ ]:
idx = 1

path = '/content/drive/MyDrive/scaffold_preprocess_dlc2/'
file_name = os.listdir(img_path)
file_name.sort()

total_black_pixel = []

for name in file_name:
    file_path = path + name;
    img = cv2.imread(file_path)
    print(file_path)
    x_Pivot = [0]
    y_Pivot = [0]
    pixel_cnt = []
  
    x_end = img.shape[0]
    y_end = img.shape[1]

    x_jump = int(x_end / 10)
    y_jump = int(y_end / 10)

    for i in range(1,10):
        cv2.line(img, (y_jump*i,0), (y_jump*i, x_end), (0,0,0))
        cv2.line(img, (0, x_jump*i), (y_end, x_jump*i), (0,0,0))

      cv2.imwrite("/content/drive/MyDrive/sliced_img_dlc2/"+name, img)
      for i in range(0,10):
        for j in range(0,10):      
            x_pivot = x_jump * j
            y_pivot = y_jump * i
            tmp_img = img[y_pivot:y_pivot+y_jump, x_pivot:x_pivot+x_jump]
            pivot_img = tmp_img
            black_pixel = np.sum(pivot_img == 0)
            pixel_cnt.append(black_pixel)
            
    total_black_pixel.append(pixel_cnt)

xy_column = []
for i in range(0,10):
    for j in range(0,10):
        xy_column.append((i,j))
df_Original = pd.DataFrame(total_black_pixel, columns=xy_column)
df_Original.to_csv("/content/drive/MyDrive/Test_data.csv",mode="w")

summary_row = []

for img_data in total_black_pixel:
    mean = np.mean(img_data)
    sum = np.sum(img_data)
    var = np.var(img_data)
    std = np.std(img_data)
    summary_row.append([sum, mean, var, std])
df_Summary = pd.DataFrame(summary_row, columns=['sum','mean','var','std'])
df_Summary.to_csv("/content/drive/MyDrive/Test_Summary.csv",mode='w')

## Processing for Reference Image

In [ ]:
thresh = 20

newImg = cv2.imread('/content/drive/MyDrive/졸논공유폴더/drawing_edit.jpg')
gray = cv2.cvtColor(newImg, cv2.COLOR_BGR2GRAY)
img2 = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)[1]
contours = cv2.findContours(img2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
plt.imshow(img2)
cv2.imwrite("asdf.jpg",img2)
x1 = [] #x_min
y1 = [] #y_min
x2 = [] #x_max
y2 = [] #y_max
for idx in  range(1, len(contours[0])): 
    ret = cv2.boundingRect(contours[0][idx]) 
    x1.append(ret[0]) 
    y1.append(ret[1]) 
    x2.append(ret[0] + ret[2]) 
    y2.append(ret[1] + ret[3])
for idx in  range(1, len(contours[1])): 
    ret = cv2.boundingRect(contours[1][idx]) 
    x1.append(ret[0]) 
    y1.append(ret[1]) 
    x2.append(ret[0] + ret[2]) 
    y2.append(ret[1] + ret[3])


delta = 0    

x1_min = min(x1)+delta
y1_min = min(y1)+delta
x2_max = max(x2)-delta
y2_max = max(y2)-delta

crop_img = newImg[y1_min:y2_max, x1_min:x2_max]

# extracted image -> Binary Image
binary_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(binary_img, thresh, 255, cv2.THRESH_BINARY)
cv2.imwrite("/content/drive/MyDrive/졸논공유폴더/drawing_crop.png",crop_img)